In [45]:

import pandas as pd
import numpy as np

df=pd.read_csv("/content/logs_dataset.csv")

print(df)

print(df.head())

print(len(df))

                         @timestamp                   _id  ip_address
0       July 8th 2019, 14:43:03.000  XswJ0msBoTGddM7vxMDB  10.1.1.285
1       July 8th 2019, 14:43:01.000  dKQJ0msB7mP0GwVzvJjz  10.1.2.389
2       July 8th 2019, 14:42:59.000  CcwJ0msBoTGddM7vtb8y  10.1.1.415
3       July 8th 2019, 14:42:57.000  bKQJ0msB7mP0GwVzrZdT   10.1.1.79
4       July 8th 2019, 14:42:55.000  L6QJ0msB7mP0GwVzpZeI   10.1.1.60
...                             ...                   ...         ...
721542  June 9th 2019, 00:00:25.000  dkWJOWsBoTGddM7vRUOR   10.1.2.66
721543  June 9th 2019, 00:00:19.000  TBuJOWsB7mP0GwVzLmol  10.1.1.249
721544  June 9th 2019, 00:00:15.000  vUWJOWsBoTGddM7vHkGD  10.1.1.200
721545  June 9th 2019, 00:00:09.000  aUWJOWsBoTGddM7vB0AR  10.1.2.432
721546  June 9th 2019, 00:00:05.000  AEWIOWsBoTGddM7v90Bw  10.1.2.156

[721547 rows x 3 columns]
                    @timestamp                   _id  ip_address
0  July 8th 2019, 14:43:03.000  XswJ0msBoTGddM7vxMDB  10.1.1.285
1  

In [46]:
df['@timestamp']=df['@timestamp'].str.replace(r'(\d+)(st|nd|rd|th)',r'\1',regex=True)
df['@timestamp']=pd.to_datetime(df['@timestamp'],format='%B %d %Y, %H:%M:%S.%f')


In [47]:
df.sort_values(['ip_address', '@timestamp'], inplace=True)

In [48]:
df['shift_time'] = df.groupby(['ip_address'])['@timestamp'].shift(1)

In [49]:
df.head()

,@timestamp,_id,ip_address,shift_time
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45


In [50]:
df['time_diff'] = (df['@timestamp']-df['shift_time']).dt.seconds//60

In [51]:
df['date'] = df['@timestamp'].dt.date

In [52]:
df['weekday'] = df['@timestamp'].dt.weekday

In [53]:
df['hour'] = df['@timestamp'].dt.hour

In [54]:
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] == 6)).astype(int)
print(df)

                @timestamp                   _id ip_address  \
721473 2019-06-09 00:06:09  DBuOOWsB7mP0GwVzhZ9U   10.1.1.1   
720483 2019-06-09 01:28:39  bB7aOWsB7mP0GwVzDY5G   10.1.1.1   
719233 2019-06-09 03:12:49  R0w5OmsBoTGddM7vayZT   10.1.1.1   
719222 2019-06-09 03:13:45  U0w6OmsBoTGddM7vRi8R   10.1.1.1   
718875 2019-06-09 03:42:39  z01UOmsBoTGddM7vuzyC   10.1.1.1   
...                    ...                   ...        ...   
3225   2019-07-08 11:49:00  3cBq0WsBoTGddM7va5TJ  10.1.2.99   
2422   2019-07-08 12:22:29  QMKJ0WsBoTGddM7vE9N1  10.1.2.99   
1704   2019-07-08 12:52:25  9pyk0WsB7mP0GwVze7sV  10.1.2.99   
1138   2019-07-08 13:15:59  B8a60WsBoTGddM7vDnqQ  10.1.2.99   
368    2019-07-08 13:47:19  M8jW0WsBoTGddM7vvplE  10.1.2.99   

                shift_time  time_diff        date  weekday  hour  is_weekend  
721473                 NaT        NaN  2019-06-09        6     0           1  
720483 2019-06-09 00:06:09       82.0  2019-06-09        6     1           1  
719233

In [55]:
df['hour_bucket'] = df['hour']//4

In [56]:
ip_addr ='ip_address'

In [57]:
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()

In [58]:
ip_counts.head()

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


In [59]:
ip_counts = ip_counts.rename(columns={'@timestamp':'total_count'})

In [60]:
daily_counts = df.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()
daily_counts = daily_counts.rename(columns={'@timestamp':'daily_counts'})
daily_counts_avg = daily_counts.groupby(ip_addr).daily_counts.median().reset_index()
daily_counts_avg.head(5)

,ip_address,daily_counts
0,10.1.1.1,40.0
1,10.1.1.100,78.0
2,10.1.1.101,40.0
3,10.1.1.106,35.5
4,10.1.1.109,42.5


In [61]:
weekend_counts = df.groupby([ip_addr, 'is_weekend'])['@timestamp'].count().reset_index()
weekend_counts

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
...,...,...,...
767,10.1.2.90,1,871
768,10.1.2.95,0,1973
769,10.1.2.95,1,895
770,10.1.2.99,0,978
